## Cleaning and combining two datasets

In [142]:
# import needed packages 
import pandas as pd


In [143]:
# Read all datasets 
indeed_data =pd.DataFrame(pd.read_csv('Indeed_data'))
stepstone_data = pd.DataFrame(pd.read_csv('StepStone_data'))

In [144]:
# Dropping the indexing column
indeed_data.drop('Unnamed: 0', axis= 1 , inplace= True)


In [145]:
# Dropping the indexing column
stepstone_data.drop('Unnamed: 0', axis= 1 , inplace= True)

In [146]:
# Combine two datasets
df = pd.concat([indeed_data,stepstone_data])

In [147]:
# See the top 5 records at the dataset
df.head()

,title,job_description
0,DATA SCIENTIST,NaN
1,Für Freelancer: Data Scientist / Optimizing ML...,NaN
2,Internship / Master Thesis – Bioprocess & Anal...,NaN
3,Data Scientist,NaN
4,"Associate Principal Scientist - Data Science, ...",NaN


In [148]:
# Check the shape of our dataset 
df.shape

(1656, 2)

In [149]:
# Check duplicates are present at in the dataset 
df.duplicated().sum()

159

In [150]:
df.drop_duplicates(inplace=True)

In [151]:
# Check for Null values 
df.isnull().sum()

title                0
job_description    274
dtype: int64

In [152]:
# Drop Null values 
df.dropna(inplace = True)

In [153]:
# Replacin new lines with spaces 
df['job_description'] = df['job_description'].str.replace('\n'," ")

## Extracting key-words from job descriptions 

In [154]:
# Creating a list with main keywords 
technologies = ['python','r','excel', 'tableau', 'power', 'bi', 'agil', 'sql', 'postgresql','sas','apache' 'spark', 
                'github', 'git','aws','microsoft azure', 'powerpoint', 'word', 'sheets', 'sqlite', 'bigquery', 'sql-datenbanken', 
                'sql-kenntnisse','r-studio','apache', 'spark','r stats','c++', 'c#', 'powerbi', 'mysql','english', 'german','java']

In [155]:
# Extracting all keywords 

# Create main dictionary to store all the keywords 
technologies_count = {}

# Create a list to add keywords which were already taked from the job posting and awoid duplication 
added_words = []

# Make a check to indicate that word "power" is detected
check = False

# The number of iterations 
n_iter = 0

# Iteration number when word "power" was detected
meeting_iter_numb = 0

# Loop through all job descriptions in the table
for description in df['job_description']:
    
    # Loop through all words in the description
    for word in description.split():

        # Lowercase words for easier comparision
        word = word.lower()
        
        # Check if the word is in all technologies list and if it's not already added
        if word in technologies and word not in added_words:

            # Increment the iteration value 
            n_iter += 1

            # Check if the word is 'bi'
            if check and word == 'bi':

                # Check if the current iteration value is one more than iteration value when the word "power" was detected
                if n_iter - meeting_iter_numb == 1:

                    # Update the count of power_bi
                    technologies_count['power_bi'] = technologies_count.get('power_bi', 0) + 1
            
            
            if word == 'power':
                # Update the check condition
                check = True

                # Store the iteration value when word 'power' was detected 
                meeting_iter_numb = n_iter
            
            # Update the count for all keywords 
            technologies_count[word] = technologies_count.get(word, 0) + 1
            added_words.append(word)

    # Clear added technologies list for each job description
    added_words.clear()

In [156]:
# Look at the technologie - count pairs 
technologies_count

{'excel': 74,
 'r': 109,
 'bi': 141,
 'german': 231,
 'english': 252,
 'power': 134,
 'sql': 197,
 'python': 363,
 'spark': 60,
 'powerbi': 19,
 'aws': 73,
 'java': 57,
 'c#': 21,
 'github': 13,
 'power_bi': 45,
 'tableau': 42,
 'powerpoint': 14,
 'sas': 17,
 'bigquery': 10,
 'apache': 43,
 'git': 37,
 'c++': 25,
 'word': 7,
 'sql-kenntnisse': 13,
 'agil': 20,
 'postgresql': 9,
 'mysql': 5,
 'sql-datenbanken': 5}

In [157]:
# Convert python dictionary to a padas dataframe
technologies_count = pd.Series(technologies_count , name  = 'Count')
technologies_count.index.name = "technologies"
technologies_count = pd.DataFrame(technologies_count.reset_index())

In [ ]:
# Create a list with all sql dialects 
sql_list = ['sql-kenntnisse', 'sql-datenbanken','postgresql','mysql','bigquery']

In [160]:
# Have a look at the dataframe 
technologies_count

,technologies,Count
technologies,,
0,excel,74
1,r,109
2,bi,141
3,german,231
4,english,252
5,power,134
6,sql,197
7,python,363
8,spark,60


In [161]:
# Find the number of sql dialects
dial_num = technologies_count[technologies_count['technologies'].isin(sql_list)]['Count'].sum()

In [163]:
# Create a new total for sql
new_total = technologies_count[technologies_count['technologies'] == 'sql']['Count'] + dial_num

In [ ]:
# Add the count of dialects to sql
technologies_count.loc[technologies_count['technologies'] == 'sql', 'Count'] = new_total

In [ ]:
# Drop sql dialects from the dataframe 
technologies_count.drop(technologies_count[technologies_count['technologies'].isin(sql_list)].index,inplace = True)

In [ ]:
technologies_count

,technologies,Count
0,excel,74
1,r,109
2,bi,141
3,german,231
4,english,252
5,power,134
6,sql,239
7,python,363
8,spark,60
9,powerbi,19


## Storing the data

In [ ]:
# Store the data as a csv file
technologies_count.to_csv('Group_cleaned')